#  Uber : Determine Best Spots for drivers

# Introduction

<!-- 
## <span style="color:red"><b>TODO & Ideas - TO BE COMMENTED</b></span>

* Make a test with time_slot_len=6 
* fig.write_image(k_AssetsDir + "/img"+ "/K-Means.png") fonctionne tout simplement pas !!!
* Splitting the notebook to avoid size problems with GitHub?
* Silhouette graphe en couteau ?

* ~~wording : Replace bin by time slot~~
* ~~wording : Pickup can be a noun or an adjective, but never a verb. Pick up is a verb phrase, but should not be used as a noun.~~
* K-Means
    * https://drlee.io/efficient-customer-segmentation-in-retail-using-kmeans-clustering-025c3961bd52
* DBSCAN 
  * ~~metric = haversine car la terre est ronde et pas plate (enfin je crois)~~
  * ~~unité de epsilon~~
  * https://blog.stackademic.com/mastering-clustering-dbscan-a880566704bc
  * https://www.sefidian.com/2022/12/18/how-to-determine-epsilon-and-minpts-parameters-of-dbscan-clustering/
  
* ~~warning K-Means~~
  * ~~set OMP_NUM_THREADS=5~~
  * ~~Get-ChildItem Env:~~
  * ~~echo $env:OMP_NUM_THREADS~~ 
  
-->


<p align="center">
<img src="./assets/ny.png" alt="drawing" width="800"/>
<p>

<!-- 
from IPython.display import display, Image
display(Image(filename='/kaggle/input/img-for-uber/ny.png')) 
-->

## Objective 
* Make sure the drivers are at the right place at the right time such that waiting time for the users does not exceed <span style="color:orange"><b>5 to 7 minutes</b></span> 
* Ideally an application would recommend hot-zones in major cities to be in at any given time of day
* Create an algorithm to find hot zones
* Visualize results on a dashboard 

## Instructions 
* Get the data from `https://www.kaggle.com/datasets/fivethirtyeight/uber-pickups-in-new-york-city`
* Save it as : ``./assets/archive.zip``
* Do <span style="color:red"><b>NOT</b></span> unzip the file
* Focus on **New-York**
* Focus on April 2014 - September 2014 timeframe
* Use cluster coordinates to pin hot zones
    * pick-up locations can be gathered into different clusters. Use cluster coordinates to pin hot zones
* Create maps with Plotly
* Start small then generalize
    * Pick one day and a given hour
    * **and then** start to generalize your approach

## Deliverables
* Have a map with hot-zones (Plotly)
* At least describe hot-zones per day of week
* Compare results with at least : K-Means and DBSCAN

## Proof of concept
After reading the instructions, here is what I have in mind :
1. The driver opens the app  
1. Based on the date and the time of the day  
1. A map shows the best spots where the driver should be (black dots in the POC below)  
    * As we shall see, the number of spots and their position depend on the date (day of the week) and time of day

<p align="center">
<img src="./assets/mock_for_drivers.png" alt="drawing" width="800"/>
<p>

<!-- display(Image(filename='/kaggle/input/img-for-uber/mock_for_drivers.png')) -->


# TOC
* [Introduction](#introduction)
* [EDA](#eda-exploratory-data-analysis)
* [Understanding the dataset](#understanding-the-dataset)
* [K-Means and DBSCAN](#identify-clusters-with-k-means-and-dbscan-methods)
* [Driver app](#driver-application-mock-up)

<!-- #### <span style="color:orange"><b>Comments :</b></span> -->
## Setup
* I'm an happy Windows 11 user
* I also use conda

In order to "play" on your host with this notebook, follow the steps below ;

1. Create a directory
1. Get the dataset from `https://www.kaggle.com/datasets/fivethirtyeight/uber-pickups-in-new-york-city`
1. Save it as : ``./assets/archive.zip``
    * Do <span style="color:red"><b>NOT</b></span> unzip the file
1. Save a copy of this notebook in the directory
1. Open a terminal in the directory

```powershell
conda create --name kaggle_uber python=3.13 -y
conda activate kaggle_uber
conda install numpy pandas seaborn plotly matplotlib scikit-learn nbformat -c conda-forge -y
code .
```

6. Once in VSCode
* Comment the line `pio.renderers.default = "iframe"` in the prelude below
    * It is mandatory in Kaggle only, NOT in VSCOode
* In the cell below, comment the line `k_AssetsDir=...` and uncomment the one saying `k_AssetsDir = Path("./assets")`
* Search for and comment the line `if False:`
* Run this notebook
* #### <span style="color:red"><b>ATTENTION :</b></span> 
    * If you commit this notebook on **GitHub** double check its size **BEFORE** to commit  
    * Under VSCode, "Clear All Outputs" if needed (I suspect Plotly to keep way too much data)  
    * `Get-ChildItem ./ -recurse | where-object {$_.length -gt 100000000} | Sort-Object length | ft fullname, length -auto`

In [ ]:
# prelude

# avoid warnings with K-Means : OpenMP (open multi processing) memory leaks under windows blablabla...
import platform
system = platform.system()
if system == "Windows":
  import os
  os.environ["OMP_NUM_THREADS"] = "2"
else:
  None


import zipfile
import datetime 
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from pathlib import Path
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder         

# -----------------------------------------------------------------------------
k_AssetsDir     = Path("./assets")
k_Gold          = 1.618              # gold number for ratio
k_Width         = 12
k_Height        = k_Width/k_Gold
k_WidthPx       = 1024
k_HeightPx      = k_WidthPx/k_Gold
k_random_state  = 0
k_Subset_Size   = 50_000             # size of the subset to speedup K-Means computing   
k_Time_Slot_Len = 10                 # length, in minutes, of each time slot  
k_Months        = ["apr", "may", "jun", "jul", "aug", "sep"] # months of interest

# -----------------------------------------------------------------------------
# Kaggle only
# pio.renderers.default = "iframe"

In [ ]:
# -----------------------------------------------------------------------------
def quick_View(df: pd.DataFrame) -> pd.DataFrame:

    """
    Generates a summary DataFrame for each column in the input DataFrame.

    This function analyzes each column in the given DataFrame and creates a summary that includes
    data type, number of null values, percentage of null values, number of non-null values, 
    number of distinct values, min and max values, outlier bounds (for numeric columns),
    and the frequency of distinct values.

    Args:
        df (pd.DataFrame): The input DataFrame to analyze.

    Returns:
        pd.DataFrame: A DataFrame containing the summary of each column from the input DataFrame. 
                      Each row in the resulting DataFrame represents a column from the input DataFrame
                      with the following information:
                      - "name": Column name
                      - "dtype": Data type of the column
                      - "# null": Number of null values
                      - "% null": Percentage of null values
                      - "# NOT null": Number of non-null values
                      - "distinct val": Number of distinct values
                      - "-3*sig": Lower bound for outliers (mean - 3*std) for numeric columns
                      - "min": Minimum value for numeric columns
                      - "max": Maximum value for numeric columns
                      - "+3*sig": Upper bound for outliers (mean + 3*std) for numeric columns
                      - "distinct val count": Dictionary of distinct value counts or top 10 values for object columns
    """

    summary_lst = []
  
    for col_name in df.columns:
        col_dtype               = df[col_name].dtype
        num_of_null             = df[col_name].isnull().sum()
        percent_of_null         = num_of_null/len(df)
        num_of_non_null         = df[col_name].notnull().sum()
        num_of_distinct_values  = df[col_name].nunique()

        if num_of_distinct_values <= 10:
            distinct_values_counts = df[col_name].value_counts().to_dict()
        else:
            top_10_values_counts    = df[col_name].value_counts().head(10).to_dict()
            distinct_values_counts  = {k: v for k, v in sorted(top_10_values_counts.items(), key=lambda item: item[1], reverse=True)}

        if col_dtype != "object":
            max_of_col = df[col_name].max()
            min_of_col = df[col_name].min()
            outlier_hi = df[col_name].mean() + 3*df[col_name].std()
            outlier_lo = df[col_name].mean() - 3*df[col_name].std()
        else:
            max_of_col = -1
            min_of_col =  1
            outlier_hi = -1
            outlier_lo =  1

        summary_lst.append({
            "name"                : col_name,
            "dtype"               : col_dtype,
            "# null"              : num_of_null,
            "% null"              : (100*percent_of_null).round(2),
            "# NOT null"          : num_of_non_null,
            "distinct val"        : num_of_distinct_values,
            "-3*sig"              : round(outlier_lo,2) ,
            "min"                 : round(min_of_col,2),
            "max"                 : round(max_of_col,2),
            "+3*sig"              : round(outlier_hi,2) ,
            "distinct val count"  : distinct_values_counts
        })

    df_tmp = pd.DataFrame(summary_lst)
    return df_tmp

In [ ]:
pattern = "uber-raw-data-{month}14.csv"
zip_path = k_AssetsDir/"archive.zip"
output_dir = k_AssetsDir
# output_dir.mkdir(parents=True, exist_ok=True)  

with zipfile.ZipFile(zip_path, 'r') as zip_file:
    # Iterate over all files in the archive
    for file_name in zip_file.namelist():
        # Check if the file matches the pattern for any month in the list
        if any(file_name == pattern.format(month=month) for month in k_Months):
            # Extract the matching file to the output directory
            zip_file.extract(file_name, output_dir)
            # print(f"Extracted: {file_name}")


# EDA (Exploratory Data Analysis)

## About the files available

In [ ]:
df_tmp = pd.read_csv(k_AssetsDir/"uber-raw-data-apr14.csv", nrows=5)
display (df_tmp)

### <span style="color:orange"><b>Comments :</b></span>

* We cannot use the column named `Base` since we cannot relate it to either a borough or a zone
* In files from 2014, we drop the column ``Base`` and only keep ``Date/Time``, ``Lat`` and ``Lon``

## Building the dataset

### <span style="color:orange"><b>Note :</b></span>

* Below it is important to notice the feature engineering which is done while the dataset is built
* Indeed some columns a extracted from the ``date_time`` features
* More important. A ``time_slot`` feature is calculated. More information are given below.


In [ ]:
def uber_preprocessor(df):
  df.columns = df.columns.str.lower()
  df.columns = df.columns.str.replace("/", "_")

  # df["base"] = df["base"].astype("category")
  df.drop(columns="base", inplace=True)

  df["date_time"] = pd.to_datetime(df["date_time"])     # dayfirst=True
  df["year"] = df["date_time"].dt.year
  df["month"] = df["date_time"].dt.month
  df["day"] = df["date_time"].dt.day
  df["weekday"] = df["date_time"].dt.weekday            # 0 monday, 1 tuesday...
  df["hour"] = df["date_time"].dt.hour
  df["minute"] = df["date_time"].dt.minute
  df.drop(columns="date_time", inplace=True)

  # may be we could drop hour and min. We will see how it goes 
  df["time_slot"] = (df["hour"]*60 + df["minute"])//k_Time_Slot_Len
  # df.drop(columns="hour", inplace=True)
  # df.drop(columns="minute", inplace=True)

  return df

### Load data and features engineering

In [ ]:
months = ["apr"]
# months = k_Months            # uncomment/comment this line to take all the months available into account 

df = pd.DataFrame()
for month in months:
  df_tmp = pd.read_csv(k_AssetsDir/f"uber-raw-data-{month}14.csv")
  df = pd.concat([df, df_tmp])

df = uber_preprocessor(df)

# display(df.sort_values(by="time_slot", ascending=False))
display(df.head(10))

### Remove duplicates

In [ ]:
print(f"Percentage of duplicates before cleaning : {(df.duplicated().sum()  / len(df)) * 100:.2f}%")
df.drop_duplicates(inplace=True)


#### <span style="color:orange"><b>Comments :</b></span>

In the dataset :
* No more duplicates
* A ``time_slot`` is a group of ``k_time_slot_len`` minutes (see the value of ``k_time_slot_len`` in the ``prelude`` cell of this notebook)
    * The ``time_slot`` feature is an index indicating the ``time`` at which the observation took place.
    * In each day, from 00H00 to 23H59, there are $\frac{23*60+59}{\text {k-time-slot-len} }$ ``time_slots`` 
    * This feature might be useful later since the users are not willing to wait more that 5-7 minutes (the length of a time slot)  

## EDA on the featured dataset

In [ ]:
print(f"The dataset consists of :")
print(f"\t{len(df.shape):>9_} dimensions")
print(f"\t{df.shape[0]:>9_} observations")
print(f"\t{df.shape[1]:>9_} features    ")

In [ ]:
df_types = pd.DataFrame ({
  "types" : df.dtypes.value_counts()
})
df_types["as_%"] = (100 * df_types["types"]/df_types["types"].sum()).round(2)

display(df_types)

In [ ]:
# df.describe(include="all").T
# df.info()

df_tmp = quick_View(df)
display(df_tmp.sort_values(by="# null", ascending=False))                 

### <span style="color:orange"><b>Comments :</b></span>
* 4.5 M observations when all months are loaded
    * 560k observations for april 2014 (see the ``# NOT null`` column)
* 0 % of null (see ``% null`` column)
* outliers ($\bar{x}$ + 3 $\sigma$ ) are in  
    * ``lat``
    * ``lon``

### Outliers

In [ ]:
col_outliers = ["lat", "lon", ]

for col in col_outliers:
  # fig = px.box(df, y=col)
  # fig.show()
  # ! Plotly can't handle the whole dataset from april to sept 
  # Let's go back to an always working safe belt (seaborn) 
  fig, ax = plt.subplots(figsize=(k_Width, k_Height))
  sns.boxplot(df, x=col)
  ax.set_title("Outliers")


How many observations are considered as outliers ?

In [ ]:
for col in col_outliers:
  upper_bound = df[col].mean() + 3*df[col].std()
  lower_bound = df[col].mean() - 3*df[col].std()
  nb_out = df.shape[0] - df[((df[col] >= lower_bound) & (df[col] <= upper_bound)) | df[col].isna()].shape[0]
  print(f"{col} have {nb_out:>6_} outliers ({100*nb_out/df.shape[0]:.2} %)")

#### <span style="color:orange"><b>Comments :</b></span>
* Since the percentage of outliers is low 
* In first approximation, we <span style="color:red"><b>remove</b></span> outliers from the dataset


In [ ]:
# -----------------------------------------------------------------------------
def remove_Outliers_Sigma(df, column):
    mean_col = df[column].mean()
    sigma_col = df[column].std()

    lower_bound = mean_col - 3 * sigma_col
    upper_bound = mean_col + 3 * sigma_col
    df = df[((df[column] >= lower_bound) & (df[column] <= upper_bound)) | df[column].isna()]
    return df


In [ ]:
print(f"Before outliers removal : {df.shape}")
for col in col_outliers:
    df = remove_Outliers_Sigma(df, col)
print(f"After  outliers removal : {df.shape}")

df_tmp = quick_View(df)
display(df_tmp.sort_values(by="# null", ascending=False))                 


# Understanding the dataset

* At this point I consider the EDA is done
* However, I need to gain a better understanding of what the data says

## Focus on one month

In [ ]:
year      = 2014                                                   
month     = 4                                                       # select another month if needed (at least, the month must be in df)
day       = 9                                                      
hour      = 7

date_obj = datetime.datetime(year, month, day)                      # day is mandatory to create a date object but it is not yet used
date_string = date_obj.strftime("%Y-%B")                            # date_string is used in the title of the graphs

# print(date_string)
# print(date_obj.strftime('%A'))

In [ ]:
# From the dataframe df we extract the selected month of the year of study (see the previous cell) 
df_full_month  = df[(df["month"] == month) & (df["year"] == year)] 

# ! Comment the next line if you want to take into account all the pick-up of the month of study  
# Plotly may be then slow or unable to display all the data
# In addition, some processings may take a while
df_month = df_full_month.sample(k_Subset_Size)

df_month.describe().T

#### <span style="color:orange"><b>Comments :</b></span>

* <span style="color:red"><b>ATTENTION :</b></span> Subset. To help Plotly and speed up the data processing, we use a sample of the pickups of the month of study  
* Double check the value of `count` in the table above


### Pickups during the month

In [ ]:
fig = px.histogram(
  df_month, 
  x="day",
  height = k_HeightPx,
  width = k_WidthPx,
  title = f"{date_string} - Pickups per day",
)
fig.show()

#### <span style="color:orange"><b>Comments :</b></span>

* The graph here above shows pickups per day during the month  
* It seems there is a kind of cycle along the weeks

### Breakdown of the pickups per day of the week

In [ ]:
fig = px.histogram(
  df_month, 
  x="weekday", 
  height = k_HeightPx, 
  width = k_WidthPx,
  title = f"{date_string} - Pickups per day (0=Monday)",
)
fig.show()


#### <span style="color:orange"><b>Comments :</b></span>

* The weekly pattern is confirmed over the month


In [ ]:
fig = px.histogram(
  df,                     # ! df NOT df_month
  x="weekday", 
  height = k_HeightPx, 
  width = k_WidthPx,
  title = "Apr-Sept 2014 - Pickups per day (0=Monday)",
)
fig.show()


#### <span style="color:orange"><b>Comments :</b></span>

* The weekly pattern is confirmed over the april-sept period of 2014
* Most active days of the week : Wenesday-Friday
* Least active days of the week : Sunday-Monday 
* <span style="color:orange"><b>Consequences :</b></span> 
    * Since the daily volume pattern is the same whatever the month 
    * This means that we will have to take the day of the week (monday, tuesday...) into account when looking for hot spots. 
    * Indeed, if on Wednesday there are twice as many pickups as on Monday, we can assume that the clusters will be of different density and distributed differently.  


In [ ]:
# Kernel density estimate (KDE) represents the data using a continuous probability density curve 

plt.figure(figsize=(k_Width, k_Height))
ax = sns.kdeplot(data = df_month, x = 'hour', fill = True, hue = 'weekday', palette = 'coolwarm', alpha = 0.2) 
ax.set_title(f"{date_string} - KDE plots pickups of each day vs hour")
ax.set_ylabel('Density of pickups')
ax.set_xticks([i for i in range(25)]) # using bw_adjust= low value ??? and cut=None does'nt work
plt.show()

#### <span style="color:orange"><b>Comments :</b></span>

* Saturday and Sunday show a pick of activities between 0 and 2AM
* Otherwise all the other days show 2 picks around 7AM and 6PM 
* The density helps to realize why friday (day 4) is so important : not only the peak in the afternoon is high but it is also very large 
* <span style="color:orange"><b>Consequence :</b></span> 
    * Since the hourly pattern depend on the day of the week (monday, tuesday...) 
    * We will have to take it into account when looking for hot spots. 
    * Indeed, hot spots at 1 AM saturday are, for sure, different than hot spots at 1 AM monday.

### Localization of the pickups during the month

In [ ]:

# it is safer to sort the values to be used as an animation_frame 
df_month = df_month.sort_values(by= "day", ascending=True)

fig = px.density_mapbox(
  df_month,
  lat="lat",
  lon="lon",
  animation_frame='day', 
  mapbox_style="carto-positron",
  radius=3,
  zoom = 10.0,
  height = k_HeightPx,
  width = k_WidthPx,
  title= f"{date_string} - Localized pickups per day over the selected month"
)
fig.show()

#### <span style="color:orange"><b>Comments :</b></span>

* The more pickups on one spot, the higher the "temperature" of the spot
* "temperature" obviously refers to the number of pickups in the day
* The view is per day along the month of study
* At this point there is no information about timing along the day
* In April 2014, 21 was a monday. We can "see" the previously identified pattern. Indeed 21 (monday) is "cooler" than 22 (tuesday) which is "cooler" than 23 (wednesday)

<p align="center">
<img src="./assets/hotspots.png" alt="drawing" width="800"/>
<p>


#### <span style="color:orange"><b>Comments :</b></span>

* No matter the day of the month, the areas here below are always on the top of the list : 
    * LaGuardia
    * Brooklyn
    * Greenpoint
    * Manhattan


### Localization of the pickups during the day (per hour, no matter the day)

In [ ]:
# it is safer to sort the values to be used as an animation_frame 
df_month = df_month.sort_values(by= "hour", ascending=True)

fig = px.density_mapbox(
  df_month,
  lat="lat",
  lon="lon",
  animation_frame='hour', 
  mapbox_style="carto-positron",
  radius=3,
  zoom = 10.0,
  height = k_HeightPx,
  width = k_WidthPx,
  title= f"{date_string} - Localized pickups per hour over the selected month"
)
fig.show()

#### <span style="color:orange"><b>Comments :</b></span>

* We look at all the days of the month at once
* We look how the pickups go per hour during an "average day"
* Brooklyn and LaGuardia slow down their pickups from 1AM to respectively 4 and 5AM
* On the other hand Manhattan and Greenpoint never stop their activites (<span style="color:orange"><b>NY, the city that never sleeps ?</b></span>)
* We must understand we <span style="color:red"><b>CANNOT</b></span> use these data to show to the drivers the spots. Indeed :
    * All days are considered equal and we know weekend != workday (for example)
    * The time resolution is not good enough. The data show were the pickups take place in the next hour while we need n-minute resolution 

## Focus on one day of the week

* Monday, tuesday...

In [ ]:
# remember the day is the n ith day of the month
# Earlier we had
# year      = 2014                                                   
# month     = 4                                                      
# day       = 9                                                      
# hour      = 7

date_obj = datetime.datetime(year, month, day)
date_string = date_obj.strftime("%Y-%B-%a")          # https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
# print (date_obj)
# print(date_obj.strftime("%A"))

In [ ]:
df_weekday = df_month[df_month["weekday"]==date_obj.weekday()]
df_weekday.describe().T

### Breakdown of the pickups over one weekday (hourly resolution)

In [ ]:
plt.figure(figsize=(k_Width, k_Height))
ax = sns.kdeplot(data = df_weekday, x = 'hour', fill = True, hue = 'weekday', palette = 'coolwarm', alpha = 0.2) 
ax.set_title(f"KDE plots pickups of all {date_obj.strftime('%A')} of {date_obj.strftime('%B %Y')} vs hour")
ax.set_ylabel('Density of pickups')
ax.set_xticks([i for i in range(25)]) # using bw_adjust= low value ??? and cut=None does'nt work
plt.show()

#### <span style="color:orange"><b>Comments :</b></span>

* This graph is consistent with the previous KDE plot

### Localization of the pickups over one weekday (hourly resolution)

In [ ]:
# it is safer to sort the values to be used as an animation_frame 
df_weekday = df_weekday.sort_values(by= "hour", ascending=True)

fig = px.density_mapbox(
  df_weekday,
  lat="lat",
  lon="lon",
  animation_frame='hour', 
  mapbox_style="carto-positron",
  radius=5,
  zoom = 10.0,
  height = k_HeightPx,
  width = k_WidthPx,
  title= f"Localized pickups for all {date_obj.strftime('%A')} of {date_obj.strftime('%B %Y')} during the next hour"
)

fig.show()

#### <span style="color:orange"><b>Comments :</b></span>

* Keep in mind that to help Plotly and speed up data processing, we work on a subset of the data.
* Bear in mind that the graph shows where the pickups will take place in the next hour for every hour of the day
* With some adjustment in zooming and panning, we can observe that at 6AM, the area of Soho (south Manhattan) and east and west side of the middle of Central Park "lite up" synchronously
* Then the center of Manhattan lites up
* Throughout the day, the “fire” spreads and covers the whole of Manhattan
* We can see where the peak of 6 PM takes place


### Localization of the pickups over one weekday (time slot resolution)

In [ ]:
# it is safer to sort the values to be used as an animation_frame 
df_weekday = df_weekday.sort_values(by= "time_slot", ascending=True)

fig = px.density_mapbox(
  df_weekday,
  lat="lat",
  lon="lon",
  animation_frame='time_slot', 
  mapbox_style="carto-positron",
  radius=10,                          # ! radius changed from 3 to 10
  zoom = 10.0,
  height = k_HeightPx,
  width = k_WidthPx,
  title= f"Localized pickups for all {date_obj.strftime('%A')} of {date_obj.strftime('%B %Y')} ({k_Time_Slot_Len} minutes time resolution)",
)

fig.show()

#### <span style="color:orange"><b>Comments :</b></span>

* This is the same graph but with a higher time resolution
* Again, keep in mind that 
    * to help Plotly and speed up data processing, we work on a subset of the data
    * the graph shows where the pickups will take place within the next n minutes, for every time slot of the day. For the value of n see the value of ``k_time_slot`` in the prelude cell of this notebook or at the title of the graph


Let's make a try with a ``scatter_mapbox()`` instead of ``density_mapbox()``

In [ ]:
fig = px.scatter_mapbox(
  df_weekday, 
  lat="lat", 
  lon="lon", 
  # opacity = 0.8,
  animation_frame='time_slot', 
  mapbox_style = "carto-positron",  
  zoom = 10.0,
  height = k_HeightPx,
  width = k_WidthPx,
  title = f"Localized pickups for all {date_obj.strftime('%A')} of {date_obj.strftime('%B %Y')} ({k_Time_Slot_Len} minutes time resolution)",
)
fig.show()

### How do pick-up times and locations differ between Sunday and Monday?

In [ ]:
df_sunday  = df_month[(df_month["weekday"] == 6) ] 

# it is safer to sort the values to be used as an animation_frame 
df_sunday = df_sunday.sort_values(by= "hour", ascending=True)

fig = px.density_mapbox(
  df_sunday,
  lat="lat",
  lon="lon",
  animation_frame='hour', 
  mapbox_style="carto-positron",
  radius=3,
  zoom = 10.0,
  height = k_HeightPx,
  width = k_WidthPx,
  title= f"Localized pickups per hour over all the Sunday of {date_obj.strftime('%B %Y')}"
)
fig.show()

In [ ]:
df_monday  = df_month[(df_month["weekday"] == 0)] 

# it is safer to sort the values to be used as an animation_frame 
df_monday = df_monday.sort_values(by= "hour", ascending=True)

fig = px.density_mapbox(
  df_monday,
  lat="lat",
  lon="lon",
  animation_frame='hour', 
  mapbox_style="carto-positron",
  radius=3,
  zoom = 10.0,
  height = k_HeightPx,
  width = k_WidthPx,
  title= f"Localized pickups per hour over all Monday of {date_obj.strftime('%B %Y')}"
)
fig.show()

#### <span style="color:orange"><b>Comments :</b></span>

* At 0 AM we can "clearly see" how Sunday differs from Monday (whether people go out at night or not)
* Same thing at 6 AM or 7 AM (day off vs working day)
* This is consistent with the KDE graph used earlier
* <span style="color:orange"><b>Consequence :</b></span> we should understand that the clusters are ``weekday`` and ``hour`` dependant

## Focus on one hour from one specific ``weekday``

* Since the pattern is `weekday` dependant
* We look at the same hour for all the same `weekday` of the same month

In [ ]:
date_obj = datetime.datetime(year, month, day, hour)
date_string = date_obj.strftime("%Y-%B-%a-%HH") # https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
print(date_string)

In [ ]:
df_hour = df_weekday[df_weekday["hour"]==hour]
df_hour.describe().T

In [ ]:
fig = px.scatter_mapbox(
  df_hour, 
  lat="lat", 
  lon="lon", 
  opacity = 0.5,
  mapbox_style = "carto-positron",  
  zoom = 10.0,
  height = k_HeightPx,
  width = k_WidthPx,
  title = f"Localized pickups for all {date_obj.strftime('%A')} of {date_obj.strftime('%B %Y')} at {date_obj.strftime('%H')}H during the next hour",
)
fig.show()

#### <span style="color:orange"><b>Comments :</b></span>

* At this time of the day (7AM) we can see that : 
    * LaGuardia   
    * East and west side of central park
    * South Manhattan & Greenpoint

are up and running

* Brooklyn is waking up


## Focus on a n-minute time slot from one specific ``weekday``

In [ ]:
# it is safer to sort the values to be used as an animation_frame 
df_hour = df_hour.sort_values(by= "time_slot", ascending=True)

fig = px.scatter_mapbox(
  df_hour, 
  lat="lat", 
  lon="lon", 
  opacity = 0.5,
  animation_frame='time_slot', 
  mapbox_style = "carto-positron",  
  zoom = 10.0,
  height = k_HeightPx,
  width = k_WidthPx,
  title = f"Localized pickups for all {date_obj.strftime('%A')} of the dataset at {date_obj.strftime('%H')}H during the next hour with {k_Time_Slot_Len} minutes resolution",
)
fig.show()

#### <span style="color:orange"><b>Comments :</b></span>

* The display of the pickups in n-minute batches between 7am and 8am seems difficult to interpret
* This might be due to the fact that, in order to help Plotly and speed up data processing, we work on a subset of the data

## Focus on a n-minute time slot from  one specific ``weekday`` of the initial dataset

* The idea is to increase the number of observations so
    * From the initial dataset...
    * Over at most, one year period...
    * We select observations from the same weekday at the same hour

In [ ]:
df_all_same_weekday_same_hour  = df[(df["weekday"] == date_obj.weekday()) & (df["year"] == year) & (df["hour"]==hour)] 
df_all_same_weekday_same_hour.describe().T

In [ ]:
# it is safer to sort the values to be used as an animation_frame 
df_all_same_weekday_same_hour = df_all_same_weekday_same_hour.sort_values(by= "time_slot", ascending=True)

fig = px.scatter_mapbox(
  df_all_same_weekday_same_hour, 
  lat="lat", 
  lon="lon", 
  opacity = 0.5,
  animation_frame='time_slot', 
  mapbox_style = "carto-positron",  
  zoom = 10.0,
  height = k_HeightPx,
  width = k_WidthPx,
  title = f"Localized pickups for all {date_obj.strftime('%A')} at {date_obj.strftime('%H')}H during the next hour with {k_Time_Slot_Len} minutes resolution",
)
fig.show()

#### <span style="color:orange"><b>Comments :</b></span>

* Compare to the previous graph, we now have 10 times more observations
* Even with 10-minute time slots it seems we can "see" some clusters
* We can groups observations this way because, thanks to the EDA, we now know the pickup are ``weekday`` dependant AND ``time`` dependant


# Identify clusters with `K-Means` and `DBScan` methods.

We start from where we are:
* a specific day of the week (wednesday for example)
* an hour (7 AM for example)
    
We want to identify the areas where a customer is likely to call in the next **10 minutes**.    


## K-Means

We're looking for the number **k** of clusters so that the clusters are spread out and have a high density. We'll use:

* **Elbow Method**:
    * We plot WCSS (Within-Cluster Sum of Squares) vs. **k**.
    * WCSS is defined as the sum for all cluster of squared distances between the centroids and each point.
    * We'll choose **k** such that adding an additional cluster no longer significantly improves the WCSS. This is typically represented as a bend or "elbow" in the WCSS curve when plotted against **k**.

* **Silhouette Score**:
    * The silhouette score measures the quality of the clustering, taking into account both 
        1. cohesion (how similar the points in the same cluster are) 
        1. separation (how distinct the clusters are from each other). 
    * It ranges from -1 to 1 where 
        1. a value close to 1 indicates that points are well-clustered and clearly separated from other clusters
        1. a value near 0 indicates that clusters are close to each other
        1. a negative value suggests that points may be incorrectly assigned. 
    
    * The optimal **k** maximizes the silhouette coefficient, meaning the clusters are well-formed (well-separated and compact).

* In practice, we select a **k** that is at the "elbow" in the WCSS curve and also corresponds to a high silhouette coefficient.
* The silhouette coefficient is a measure that evaluates the quality of clustering, taking into account both cohesion (how similar the points in the same cluster are) and separation (how distinct the clusters are from each other).
 
 


In [ ]:
# At this stage the 7AM is still hardcoded as 42 (42 = 7 * 60/10 + 0 minutes)
# First let's make a copy of the features of interest
df_for_kmeans = df_all_same_weekday_same_hour[["lat", "lon", "time_slot"]]
# filter and only keep the observations for the next 10 minutes after 7AM
df_for_kmeans = df_for_kmeans[df_for_kmeans["time_slot"]==42]
df_for_kmeans.drop(columns="time_slot", inplace=True)
df_for_kmeans.describe().T

In [ ]:
fig = px.scatter_mapbox(
  df_for_kmeans, 
  lat="lat", 
  lon="lon", 
  opacity = 0.5,
  mapbox_style = "carto-positron",  
  zoom = 10.0,
  height = k_HeightPx,
  width = k_WidthPx,
  title = f"Localized pickups {date_obj.strftime('%A')} at {date_obj.strftime('%H')}H during the next {k_Time_Slot_Len} minutes",
)
fig.show()

Draw wcss and slihouette graph

In [ ]:
wcss = []
sil=[]
k=[]
for i in range(2, 20): # Z! 2 to 11
  kmeans = KMeans(n_clusters=i, random_state = k_random_state, n_init='auto') # default init='k-means++', n_init='auto' avoid warning
  kmeans.fit(df_for_kmeans)
  wcss.append(kmeans.inertia_)
  sil.append(silhouette_score(df_for_kmeans, kmeans.predict(df_for_kmeans)))
  k.append(i)

df_for_k_choice = pd.DataFrame({'k':k, 'wcss':wcss, 'sil':sil}).set_index('k')

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_for_k_choice.index,
        y=df_for_k_choice['wcss'],
        mode='lines+markers',
        name='WCSS',             # add the name to the legend
        line=dict(color='blue')  # color of the line
    )
)

fig.add_trace(
    go.Bar(
        x=df_for_k_choice.index,
        y=df_for_k_choice['sil'],
        name='Silhouette Score',
        yaxis='y2',
        opacity=0.6
    )
)

fig.update_layout(
    # title_text=f"{date_string} - KMEANS - Determine optimal # of clusters (k)",
    title_text=f"Wed-[April-Sept]-2014 - KMEANS - Determine optimal # of clusters (k)",
    xaxis_title="k",
    yaxis=dict(
        title="WCSS",
    ),
    yaxis2=dict(
        title="Silhouette Score",
        overlaying='y',
        side='right'
    ),
    height=k_HeightPx,
    width=k_WidthPx,
)

fig.show()


* We choose k that lies at the elbow of the WCSS curve and also corresponds to a high value of the silhouette coefficient. 
* We select k = 10 
* We don't select k=8 because Silhouette score is lower 
* We don't select k=11 because the WCSS doesn't decrease significantly when k goes from 10 to 11.

In [ ]:
df_with_kmeans_clusters = df_all_same_weekday_same_hour[["lat", "lon", "time_slot"]]
df_with_kmeans_clusters = df_with_kmeans_clusters[df_with_kmeans_clusters["time_slot"]==42]
df_with_kmeans_clusters.drop(columns="time_slot", inplace=True)

k_optimal = 10
kmeans = KMeans(n_clusters=k_optimal, random_state = k_random_state, n_init='auto') # default init='k-means++', n_init='auto' avoid warning 
df_with_kmeans_clusters['cluster'] = kmeans.fit_predict(df_with_kmeans_clusters)


* Display les cluster in colors
* The black dots on the map are the centroids of the clusters

In [ ]:
kmeans_centroids = pd.DataFrame(kmeans.cluster_centers_, columns=['lat', 'lon'])

# force "cluster" to be NOT interpreted as a float number 
df_with_kmeans_clusters['cluster'] = df_with_kmeans_clusters['cluster'].astype(str)


# df_with_kmeans_clusters = df_with_kmeans_clusters.sort_values(by= "time_slot", ascending=True)

fig = px.scatter_mapbox(
  df_with_kmeans_clusters, 
  lat="lat", 
  lon="lon", 
  opacity = 0.5,
#   animation_frame='time_slot', 
  mapbox_style = "carto-positron",  
  color='cluster',
  color_discrete_sequence=px.colors.qualitative.Plotly,                             # Use a discrete color sequence : Plotly D3 Set1 Pastel1 Category10
  # color_discrete_sequence=['red', 'blue', 'green', 'purple', 'orange', 'black'],  # Custom color sequence
  zoom = 10.0,
  height = k_HeightPx,
  width = k_WidthPx,
  title = f"K-Means - Localized pickups {date_obj.strftime('%A')} at {date_obj.strftime('%H')}H within the next {k_Time_Slot_Len} minutes"
)

# fig.add_trace(px.scatter_mapbox(
#     kmeans_centroids,
#     lat='lat',
#     lon='lon',
#     mapbox_style="carto-positron"
# ).data[0])

fig.add_trace(go.Scattermapbox(
    lat=kmeans_centroids['lat'],
    lon=kmeans_centroids['lon'],
    mode='markers',
    marker=go.scattermapbox.Marker(
        size=10,           
        color='black',       
        opacity=0.8        
    ),
    name='Centroids'       
))

fig.show()

# Simply does NOT work
# conda install -c conda-forge python-kaleido
# fig.write_image(k_AssetsDir + "/img"+ "/K-Means.png")


### <span style="color:orange"><b>Comments :</b></span>
THE thing to understand is :
* Even if a cluster is very large (cluster 0, Manhattan for example)
* If the cab is in the cluster
* Then the density of the cluster (in terms of customers who will call) is such that the cab will be able to serve a customer in less than ``k_time_slot`` min.
* Example 
    * Large Manhattan cluster
    * The cab is on the south side
    * There may be a call from a customer on the north side of Manhattan that the cab won't be able to serve in less than 5 min...
    * <span style="color:orange"><b>BUT</b></span>, we are also sure that there will be a call from a customer in the SOUTH of Manhattan that the driver will be able to serve in less than 5 min.


### <span style="color:orange"><b>Comments :</b></span>


K-Means can be used to determine from which clusters a customer is likely to call in the next 10 minutes.

So we could imagine that the driver presses a button of an on an application. From that moment on, the application could :
1. finds the day of the week (`weekday`)
1. finds the time
1. invoque ``kmean.fit()`` 

However, this is where we reach one of the <span style="color:red"><b>limitations</b></span> of K-Means. Indeed, one have to <span style="color:red"><b>choose k</b></span> before the application can go on invoking ``kmeans.fit_predict()`` and display the clusters on a map. Yes we could automate one way or another the way to find optimal k. For example, this could be done using the slope of WCSS and checking how the Silhouette score evolve.



## DBSCAN

In [ ]:
# At this stage the 7AM is still hardcoded as 42 (42 = 7 * 60/10 + 0 minutes)
# First let's make a copy of the features of interrest
df_for_dbscan = df_all_same_weekday_same_hour[["lat", "lon", "time_slot"]]
# filter and only keep the observations for the next 10 minutes after 7AM
df_for_dbscan = df_for_dbscan[df_for_dbscan["time_slot"]==42]
df_for_dbscan.drop(columns="time_slot", inplace=True)
df_for_dbscan.describe().T

In [ ]:
fig = px.scatter_mapbox(
  df_for_dbscan, 
  lat="lat", 
  lon="lon", 
  opacity = 0.5,
  mapbox_style = "carto-positron",  
#   color='cluster',
  zoom = 10.0,
  height = k_HeightPx,
  width = k_WidthPx,
  title = f"Localized pickups {date_obj.strftime('%A')} at {date_obj.strftime('%H')}H within the next {k_Time_Slot_Len} minutes"
)
fig.show()

In [ ]:

# ! Conversion radian
df_for_dbscan_rad = np.deg2rad(df_for_dbscan)
df_for_dbscan_rad.describe().T

In [ ]:
# df_for_dbscan = np.deg2rad(df_for_dbscan[["lat", "lon"]])
# df_for_dbscan.describe().T

In [ ]:
# avg radius of Earth in kilometers
# kms_per_radian = 6371.0088         

# pickups within 3 km
# epsilon = 3000/ kms_per_radian

epsilon = 0.0001 # in radian because lat and lon have been converted to radian (0.05 d° => 0.00087 rad)
                 # 0.0001 rad = 637 m
min_samples = 4  # heuristic 2 * number_of_dimensions => 2*2=4


# ! metric='haversine' => https://blog.stackademic.com/mastering-clustering-dbscan-a880566704bc
dbscan = DBSCAN(eps=epsilon, min_samples=min_samples, metric='haversine', algorithm="auto")       # algorithm='ball_tree', metric="euclidean"
dbscan.fit(df_for_dbscan_rad)                                                                     # Y a pas de predict

# df_hour_extended = df_hour.copy()
df_for_dbscan['cluster'] = dbscan.labels_

# df_tmp = pd.DataFrame(db.labels_)
# df_tmp[0].value_counts()

print(set(dbscan.labels_))
print(len(set(dbscan.labels_)))


In [ ]:
# force "cluster" to be NOT interpreted as a float number 
df_for_dbscan['cluster'] = df_for_dbscan['cluster'].astype(str)

fig = px.scatter_mapbox(
  df_for_dbscan[df_for_dbscan['cluster']!="-1"],  # do not display anomalies
  lat="lat", 
  lon="lon", 
  opacity = 0.5,
  mapbox_style = "carto-positron",  
  color='cluster',
  color_discrete_sequence=px.colors.qualitative.Plotly,                             # Use a discrete color sequence : Plotly D3 Set1 Pastel1 Category10
  # color_discrete_sequence=['red', 'blue', 'green', 'purple', 'orange', 'black'],  # Custom color sequence
  zoom = 10.0,
  height = k_HeightPx,
  width = k_WidthPx,
  title = f"DBSCAN - Localized pickups {date_obj.strftime('%A')} at {date_obj.strftime('%H')}H within the next {k_Time_Slot_Len} minutes"
)
fig.show()

In [ ]:
# How to select epsilon ?
# https://www.sefidian.com/2022/12/18/how-to-determine-epsilon-and-minpts-parameters-of-dbscan-clustering/

neigh = NearestNeighbors(n_neighbors=4)             # heuristic 2 * number_of_dimensions => 2*2=4
nbrs = neigh.fit(df_for_dbscan_rad)
distances, indices = nbrs.kneighbors(df_for_dbscan_rad) # distance and indices of the 4 nearest neighbors
# print(distances.shape)
# print(distances)

# Sort distances by ascending order
distances = np.sort(distances[:, 3])                   # extract & sort the 4th nearest neighbor
# print(distances.shape)
# print(distances)

plt.plot(distances)
plt.xlabel('Points index')
plt.ylabel('Distance to 4th nearest neighbor')
plt.title('k-distance Graph')

plt.ylim(0, 0.0005)
plt.xlim(600, 850)                                   # "zoom" in the region of interest

# xv and yh are "hand adjusted" to cross at tje "elbow" nicely
xv = 765
yh = 0.00012                                         # epsilon
plt.axvline(x=xv, color='red', linestyle='--')
plt.axhline(y=yh, color='red', linestyle='--')

plt.show()

In [ ]:
# Could we use the derivative to automate the detection of the previous elbow ?
# print(distances)
derivative = [distances[i + 1] - distances[i] for i in range(len(distances) - 1)]
# print(differences)

plt.plot(derivative)
plt.xlabel('Points')
plt.ylabel('Distance to 4th nearest neighbor')
plt.title('Derivative of the k-distance Graph')

plt.ylim(0, 0.0002)
plt.xlim(600, 850)

plt.show()

# Driver application mock-up

In [ ]:
# In the mock-up this is where the driver enters the day of the week and the time
# Yes, the application could determine automatically the weekday and the time

year    = 2014
weekday = 0
time    = "07:30"

# weekday = 2
# time = "07:00"

# weekday = 0
# time = "01:00"



In [ ]:
months = ["apr", "may", "jun", "jul", "aug", "sep"]            
week_days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
day_of_week = week_days[weekday]
hour, minute = map(float, time.split(":"))

df = pd.DataFrame()
for month in months:
  df_tmp = pd.read_csv(f"assets/uber-raw-data-{month}14.csv")
  df = pd.concat([df, df_tmp])

df = uber_preprocessor(df)
df.drop_duplicates(inplace=True)

col_outliers = ["lat", "lon"]
for col in col_outliers:
    df = remove_Outliers_Sigma(df, col)

df_tmp = quick_View(df)
display(df_tmp.sort_values(by="# null", ascending=False))                 


In [ ]:
df_all_same_weekday_same_hour  = df[(df["weekday"] == weekday) & (df["year"] == year) & (df["hour"]==hour) & (df["minute"]==minute)] 
# df_all_same_weekday_same_hour  = df[(df["weekday"] == weekday) & (df["year"] == year) & (df["hour"]==hour)] 
# df_all_same_weekday_same_hour.describe().T

In [ ]:
df_for_kmeans = df_all_same_weekday_same_hour[["lat", "lon", "time_slot"]]
# filter and only keep the observations for the next 10 minutes after hour
df_for_kmeans = df_for_kmeans[df_for_kmeans["time_slot"]==(hour*60+minute)/k_Time_Slot_Len]
df_for_kmeans.drop(columns="time_slot", inplace=True)
# df_for_kmeans.describe().T

In [ ]:
wcss = []
sil=[]
k=[]
for i in range(2, 20): # Z! 2 to 11
  kmeans = KMeans(n_clusters=i, random_state = k_random_state, n_init='auto') # default init='k-means++', n_init='auto' avoid warning
  kmeans.fit(df_for_kmeans)
  wcss.append(kmeans.inertia_)
  sil.append(silhouette_score(df_for_kmeans, kmeans.predict(df_for_kmeans)))
  k.append(i)

df_for_k_choice = pd.DataFrame({'k':k, 'wcss':wcss, 'sil':sil}).set_index('k')

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_for_k_choice.index,
        y=df_for_k_choice['wcss'],
        mode='lines+markers',
        name='WCSS',             # Ajout du nom dans la légende
        line=dict(color='blue')  # couleur pour la ligne
    )
)

fig.add_trace(
    go.Bar(
        x=df_for_k_choice.index,
        y=df_for_k_choice['sil'],
        name='Silhouette Score',
        yaxis='y2',
        opacity=0.6
    )
)

fig.update_layout(
    title_text=f"KMEANS - Determine optimal # of clusters (k)",
    xaxis_title="k",
    yaxis=dict(
        title="WCSS",
    ),
    yaxis2=dict(
        title="Silhouette Score",
        overlaying='y',
        side='right'
    ),
    height=k_HeightPx,
    width=k_WidthPx,
)

fig.show()


In [ ]:
# Later, k_optimal will be determined automatically
# See the next cell for example
# As today you can either use 10 (seems to be working most of the time) or adjust it base on what you see on the previous graph

k_optimal = 12

In [ ]:

# ! Prototype
def select_optimal_k(wcss, silhouette_scores, k_values):
    # Elbow method: Select k where the WCSS curve bends
    # Choose k where WCSS decreases significantly slower (i.e., the "elbow")
    wcss_diffs = np.diff(wcss)                                                      # first derivative
    wcss_diffs_diffs = np.diff(wcss_diffs)                                          # second derivative 
    optimal_k = k_values[np.argmin(wcss_diffs_diffs) + 1]
    # print(f"k via elbow :{optimal_k}")
    
    # Silhouette method 
    # After k is selected via elbow, look around k +/- 1 (or 2..., see delta) and then pick k that maximizes the silhouette score within the window
    delta = 1
    lower_bound = max(0, optimal_k - delta - min(k_values))                         # make sure don't go below or above k_values range
    upper_bound = min(len(k_values) - 1, optimal_k + delta - min(k_values))  
    subset_k_values = k_values[lower_bound:upper_bound + 1]
    # print(subset_k_values)
    subset_silhouette_scores = silhouette_scores[lower_bound:upper_bound + 1]
    # print(subset_silhouette_scores)
    optimal_k = subset_k_values[np.argmax(subset_silhouette_scores)]
    
    return optimal_k

optimal_k = select_optimal_k(df_for_k_choice["wcss"], df_for_k_choice["sil"], df_for_k_choice.index)
print(f"Optimal k : {optimal_k}")


In [ ]:
df_with_kmeans_clusters = df_all_same_weekday_same_hour[["lat", "lon", "time_slot"]]
df_with_kmeans_clusters = df_with_kmeans_clusters[df_with_kmeans_clusters["time_slot"]==(hour*60+minute)/k_Time_Slot_Len]
df_with_kmeans_clusters.drop(columns="time_slot", inplace=True)

kmeans = KMeans(n_clusters=k_optimal, random_state = k_random_state, n_init='auto') # default init='k-means++', n_init='auto' avoid warning 
df_with_kmeans_clusters['cluster'] = kmeans.fit_predict(df_with_kmeans_clusters)


In [ ]:
kmeans_centroids = pd.DataFrame(kmeans.cluster_centers_, columns=['lat', 'lon'])

# force "cluster" to be NOT interpreted as a float number 
df_with_kmeans_clusters['cluster'] = df_with_kmeans_clusters['cluster'].astype(str)


# df_with_kmeans_clusters = df_with_kmeans_clusters.sort_values(by= "time_slot", ascending=True)

fig = px.scatter_mapbox(
  df_with_kmeans_clusters, 
  lat="lat", 
  lon="lon", 
  opacity = 0.5,
#   animation_frame='time_slot', 
  mapbox_style = "carto-positron",  
  color='cluster',
  color_discrete_sequence=px.colors.qualitative.Plotly,                             # Use a discrete color sequence : Plotly D3 Set1 Pastel1 Category10
  # color_discrete_sequence=['red', 'blue', 'green', 'purple', 'orange', 'black'],  # Custom color sequence
  zoom = 10.0,
  height = k_HeightPx,
  width = k_WidthPx,
  title = f"K-Means - Localized pickups {day_of_week} at {int(hour):02d}:{int(minute):02d} within the next {k_Time_Slot_Len} minutes"
)

fig.add_trace(go.Scattermapbox(
    lat=kmeans_centroids['lat'],
    lon=kmeans_centroids['lon'],
    mode='markers',
    marker=go.scattermapbox.Marker(
        size=10,           
        color='black',       
        opacity=0.8        
    ),
    name='Centroids'       
))

fig.show()

